## How to create a MLP in {tool}?

In [ ]:
import datetime

import numpy as np

from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### Toy Dataset

In [ ]:
x, y = make_classification(n_samples=1000)
y = y.astype(float)

In [ ]:
xt, xv, yt, yv = train_test_split(x, y, train_size=.8)

### PyTorch

In [ ]:
import torch
import pytorch_lightning as pl
import torch.nn.functional as F

from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
xp = torch.Tensor(xt)
yp = torch.Tensor(yt)
ds = TensorDataset(xp, yp)
dl = DataLoader(ds, batch_size=4)

In [ ]:
class TorchNN(pl.LightningModule):
    def __init__(self, lr: float = 1e-3):
        super().__init__()
        self.lr = lr
        self.nn = nn.Sequential(
            nn.Linear(20, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
    
    def forward(self, x):
        features = self.nn(x)
        return features
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.nn(x).view(-1)
        loss = F.binary_cross_entropy_with_logits(y_hat, y) # escolhi a função certa?
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).view(-1)
        val_loss = F.binary_cross_entropy_with_logits(y_hat, y)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).view(-1)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def report(self, x, y):
        y = y.detach().numpy()
        y_hat = nn.Sigmoid()(self(x)).detach()
        y_hat = y_hat.numpy().ravel().round()
        print(classification_report(y, y_hat, digits=4))

In [ ]:
nn_torch = TorchNN()
trainer = pl.Trainer(max_epochs=20)

In [ ]:
trainer.fit(nn_torch, dl)

In [ ]:
nn_torch.report(xp, yp)

Verificamos o modelo em
`tensorboard --logdir lightning_logs/`

### TensorFlow

In [ ]:
load_ext tensorboard

In [ ]:
import tensorflow as tf

In [ ]:
class FlowNN(tf.keras.Model):
    def __init__(self):
        super(FlowNN, self).__init__()
        inputs = tf.keras.layers.Input(shape=[20])
        hidden = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
        outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(hidden)
        self.model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    
    def call(self, inputs):
        return self.model(inputs)

In [ ]:
log_dir = "tensorflow_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
flow_nn = FlowNN()

In [ ]:
flow_nn.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [ ]:
flow_nn.fit(
    xt,
    yt,
    epochs=30,
    validation_data=(xv, yv),
    callbacks=[tensorboard_callback]
)

Verificamos o modelo em `tensorboard --logdir tensorflow_logs`